# Swiss Zip Code Data from die Post

In order to filter meaningful location hashtags, and destinations from MySwitzerland, addtional data such as a full list of canton names, and corrsponding regions, cities and geo locations are need. Therefore, we got two data source from Die Post and Wikipedia.

 - Swiss zip code data which is downloaded directly from Die post website. Here is the Data Source:https://swisspost.opendatasoft.com/explore/embed/dataset/plz_verzeichnis_v2/table/?disjunctive.postleitzahl
 - A table `zip_code` is created to store data after data clearning and manupulation
 - Secondly, a table `cantons` is created which is the name of 26 cantons in Switzerland. Here, we manunally entered English, German and short code of each canton. 
 

In [1]:
%config Completer.use_jedi= False

In [7]:
import requests
import re
import pandas as pd
import pprint
import json
import csv
import numpy as np

import urllib.request 
import urllib.error
from time import gmtime, strftime
import time
import datetime
from time import sleep
import psycopg2
import config
import os.path
from sqlalchemy import create_engine

In [27]:
zip_code = pd.read_csv('zip_code.csv', sep= ',')
zip_code

,Unnamed: 0,rec_art,onrp,bfsnr,plz_typ,postleitzahl,plz_zz,gplz,ortbez18,ortbez27,kanton,sprachcode,briefz_durch,plz_briefzust,plz_coff,geokoordinaten,latitude,longitude
0,1,1,104,5586,20,1000,0,1000,Lausanne,Lausanne,VD,2,130,100060,N,NaN,NaN,NaN
1,2,1,106,5586,80,1000,2,1000,Lausanne 2,Lausanne 2,VD,2,149,100200,NaN,NaN,NaN,NaN
2,3,1,114,5586,30,1000,12,1000,Lausanne 12,Lausanne 12,VD,2,114,100012,NaN,NaN,NaN,NaN
3,4,1,117,5586,30,1000,16,1000,Lausanne 16,Lausanne 16,VD,2,117,100016,NaN,NaN,NaN,NaN
4,5,1,122,5586,30,1000,22,1000,Lausanne 22,Lausanne 22,VD,2,122,100022,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5280,5281,1,9236,247,80,8010,57,8010,Zürich Briefklinik,Zürich BZ Briefklinik,ZH,1,6455,801050,NaN,NaN,NaN,NaN
5281,5282,1,9256,2763,80,4144,80,4144,Arlesheim Hub,Arlesheim Postlogistics Hub,BL,1,7184,415360,NaN,NaN,NaN,NaN
5282,5283,1,9261,6621,80,1200,30,1200,Genève Les Acacias,Genève Les Acacias,GE,2,7162,122760,NaN,NaN,NaN,NaN
5283,5284,1,9289,5402,20,1880,5,1880,Les Posses-sur-Bex,Les Posses-sur-Bex,VD,2,7666,188060,J,"46.2741595819,7.04575184165",46.2741595819,7.04575184165


In [28]:
zip_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5285 entries, 0 to 5284
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      5285 non-null   int64 
 1   rec_art         5285 non-null   int64 
 2   onrp            5285 non-null   int64 
 3   bfsnr           5285 non-null   int64 
 4   plz_typ         5285 non-null   int64 
 5   postleitzahl    5285 non-null   int64 
 6   plz_zz          5285 non-null   int64 
 7   gplz            5285 non-null   int64 
 8   ortbez18        5285 non-null   object
 9   ortbez27        5285 non-null   object
 10  kanton          5285 non-null   object
 11  sprachcode      5285 non-null   int64 
 12  briefz_durch    5285 non-null   int64 
 13  plz_briefzust   5285 non-null   int64 
 14  plz_coff        4083 non-null   object
 15  geokoordinaten  4062 non-null   object
 16  latitude        4063 non-null   object
 17  longitude       4063 non-null   object
dtypes: int64

In [29]:
del zip_code['Unnamed: 0']

In [30]:
zip_code.columns

Index(['rec_art', 'onrp', 'bfsnr', 'plz_typ', 'postleitzahl', 'plz_zz', 'gplz',
       'ortbez18', 'ortbez27', 'kanton', 'sprachcode', 'briefz_durch',
       'plz_briefzust', 'plz_coff', 'geokoordinaten', 'latitude', 'longitude'],
      dtype='object')

In [31]:
zip_code.index = np.arange(1, len(zip_code) + 1)

In [32]:
zip_code = pd.DataFrame(zip_code)
zip_code

,rec_art,onrp,bfsnr,plz_typ,postleitzahl,plz_zz,gplz,ortbez18,ortbez27,kanton,sprachcode,briefz_durch,plz_briefzust,plz_coff,geokoordinaten,latitude,longitude
1,1,104,5586,20,1000,0,1000,Lausanne,Lausanne,VD,2,130,100060,N,NaN,NaN,NaN
2,1,106,5586,80,1000,2,1000,Lausanne 2,Lausanne 2,VD,2,149,100200,NaN,NaN,NaN,NaN
3,1,114,5586,30,1000,12,1000,Lausanne 12,Lausanne 12,VD,2,114,100012,NaN,NaN,NaN,NaN
4,1,117,5586,30,1000,16,1000,Lausanne 16,Lausanne 16,VD,2,117,100016,NaN,NaN,NaN,NaN
5,1,122,5586,30,1000,22,1000,Lausanne 22,Lausanne 22,VD,2,122,100022,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5281,1,9236,247,80,8010,57,8010,Zürich Briefklinik,Zürich BZ Briefklinik,ZH,1,6455,801050,NaN,NaN,NaN,NaN
5282,1,9256,2763,80,4144,80,4144,Arlesheim Hub,Arlesheim Postlogistics Hub,BL,1,7184,415360,NaN,NaN,NaN,NaN
5283,1,9261,6621,80,1200,30,1200,Genève Les Acacias,Genève Les Acacias,GE,2,7162,122760,NaN,NaN,NaN,NaN
5284,1,9289,5402,20,1880,5,1880,Les Posses-sur-Bex,Les Posses-sur-Bex,VD,2,7666,188060,J,"46.2741595819,7.04575184165",46.2741595819,7.04575184165


In [37]:
zip_code1 = zip_code.replace(np.nan, "null", regex=True)
print(zip_code1)

      rec_art  onrp  bfsnr  plz_typ  postleitzahl  plz_zz  gplz  \
1           1   104   5586       20          1000       0  1000   
2           1   106   5586       80          1000       2  1000   
3           1   114   5586       30          1000      12  1000   
4           1   117   5586       30          1000      16  1000   
5           1   122   5586       30          1000      22  1000   
...       ...   ...    ...      ...           ...     ...   ...   
5281        1  9236    247       80          8010      57  8010   
5282        1  9256   2763       80          4144      80  4144   
5283        1  9261   6621       80          1200      30  1200   
5284        1  9289   5402       20          1880       5  1880   
5285        1  9290   5003       80          6590      68  6590   

                ortbez18                     ortbez27 kanton  sprachcode  \
1               Lausanne                     Lausanne     VD           2   
2             Lausanne 2                   

In [38]:
zip_code1.to_csv('zip_code1.csv', sep = ",")

In [40]:
# connect to postgres database
conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
cursor = conn.cursor()
conn.autocommit = True

def createZipChTable():
    cursor.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'zip_code'")
    if cursor.fetchone()[0] != 1:
        #print('create table')
        cursor.execute(f"""
        CREATE TABLE "public"."zip_code" (
        "zip_id" SERIAL PRIMARY KEY not null,
          "rec_art" INT , 
          "onrp" INT, 
          "bfsnr" INT, 
          "plz_typ" INT, 
          "postleitzahl" INT, 
          "plz_zz" INT, 
          "gplz" INT,
       "ortbez18" VARCHAR(64), 
       "ortbez27" VARCHAR(64), 
       "kanton"VARCHAR(64) not null, 
       "sprachcode" INT,
       "briefz_durch" INT, 
       "plz_briefzust" VARCHAR(64), 
       "plz_coff" VARCHAR(64), 
       "geokoordinaten" VARCHAR(255),
       "latitude" VARCHAR(64) null,
       "longitude" VARCHAR(64) null);
          """)

createZipChTable()

In [41]:
conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
cursor = conn.cursor()
conn.autocommit = True

with open('zip_code1.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
        cursor.execute(
        "INSERT INTO zip_code VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s,%s, %s,%s, %s, %s, %s,%s,%s)",
        row
    )
conn.commit()

In [8]:
kanton_ch = {
  "canton": ["Zürich", "Bern","Luzern", "Uri","Schwyz","Obwalden","Nidwalden","Glarus","Zug","Fribourg","Solothurn","Basel Stadt","Basel Land","Schaffhausen","Appenzell Ausserrhoden", "Appenzell Innerrhoden","St Gallen","Graubünden","Aargau","Thurgau","Ticino","Vaud","Valais","Neuchâtel","Genève","Jura"],
    "canton_en":["Zurich", "Berne","Luzerne", "Uri","Schwyz","Obwalden","Nidwalden","Glarus","Zug","Freiburg","Solothurn","Basel City","Basel Land","Schaffhausen","Appenzell Outer Rhodes", "Appenzell Inner Rhodes","St Gallen","Grisons","Aargau","Thurgau","Ticino","Vaud","Wallis","Neuchatel","Geneve","Jura"],
  "short": ["ZH", "BE","LU","UR","SZ","OW","NW","GL","ZG","FR","SO","BS","BL","SH","AR","AI","SG","GR","AG","TG","TI","VD","VS","NE","GE","JU"],
    "capital":["Zürich", "Bern", "Luzern", "Altdorf", "Schwyz","Sarnen","Stans","Glarus","Zug","Fribourg","Solothurn","Basel","Liestal","Schaffhausen","Herisau","Appenzell","St Gallen","Chur","Aarau","Frauenfeld","Bellinzona","Lausanne","Sitten / Sion","Neuchâtel","Genève","Delemont"]}

In [9]:
cantons = pd.DataFrame(kanton_ch)
cantons.index = np.arange(1, len(cantons) + 1)
cantons

,canton,canton_en,short,capital
1,Zürich,Zurich,ZH,Zürich
2,Bern,Berne,BE,Bern
3,Luzern,Luzerne,LU,Luzern
4,Uri,Uri,UR,Altdorf
5,Schwyz,Schwyz,SZ,Schwyz
6,Obwalden,Obwalden,OW,Sarnen
7,Nidwalden,Nidwalden,NW,Stans
8,Glarus,Glarus,GL,Glarus
9,Zug,Zug,ZG,Zug
10,Fribourg,Freiburg,FR,Fribourg


In [10]:
cantons.to_csv('cantons.csv', sep = ",", encoding='utf-8')

In [11]:
# connect to postgres database
conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
cursor = conn.cursor()
conn.autocommit = True

def createkantonTable():
    cursor.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'cantons'")
    if cursor.fetchone()[0] != 1:
        #print('create table')
        cursor.execute(f"""
        CREATE TABLE "public"."cantons" (
          "can_id" SERIAL PRIMARY KEY not null,
          "canton" VARCHAR(64) UNIQUE not null,
          "canton_en" VARCHAR(64) UNIQUE not null,
          "short" VARCHAR (64) UNIQUE not null,
          "capital" VARCHAR(64) not null);
          """)
        
createkantonTable()

In [12]:
conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
cursor = conn.cursor()
conn.autocommit = True

with open('cantons.csv', 'r') as f2:
    next(f2) # Skip the header row.
    cursor.copy_from(f2, 'cantons', sep=',')

conn.commit()